In [8]:
!pip install segmentation-models -q

import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
import segmentation_models as sm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from tqdm import tqdm

BASE_PATH = '/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle'
TRAIN_IMG_DIR = os.path.join(BASE_PATH, 'train_images')
TRAIN_MASK_DIR = os.path.join(BASE_PATH, 'train_masks')
TEST_IMG_DIR = os.path.join(BASE_PATH, 'test_images_padded')

IMG_HEIGHT = 256
IMG_WIDTH = 256
BATCH_SIZE = 16
EPOCHS = 3
BACKBONE = 'resnet34'

preprocess_input = sm.get_preprocessing(BACKBONE)

print("Configuration Set.")

class TerraSegGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_ids, img_dir, mask_dir=None, batch_size=16, img_size=(256, 256), is_train=True):
        self.image_ids = image_ids
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.batch_size = batch_size
        self.img_size = img_size
        self.is_train = is_train

    def __len__(self):
        return int(np.ceil(len(self.image_ids) / self.batch_size))

    def __getitem__(self, index):
        batch_ids = self.image_ids[index * self.batch_size : (index + 1) * self.batch_size]
        
        images = []
        masks = []
        
        for img_id in batch_ids:
            img_path = os.path.join(self.img_dir, img_id)
            img = cv2.imread(img_path)
            img = cv2.resize(img, self.img_size)
            
            img = preprocess_input(img)
            images.append(img)
            
            if self.is_train:
                mask_path = os.path.join(self.mask_dir, img_id) 
                mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
                
                mask = cv2.resize(mask, self.img_size, interpolation=cv2.INTER_NEAREST)
                
                mask = (mask > 0).astype(np.float32)
                
                mask = np.expand_dims(mask, axis=-1)
                masks.append(mask)
        
        if self.is_train:
            return np.array(images).astype(np.float32), np.array(masks).astype(np.float32)
        else:
            return np.array(images).astype(np.float32)

all_ids = os.listdir(TRAIN_IMG_DIR)
train_ids, val_ids = train_test_split(all_ids, test_size=0.2, random_state=42)

train_gen = TerraSegGenerator(train_ids, TRAIN_IMG_DIR, TRAIN_MASK_DIR, BATCH_SIZE, (IMG_HEIGHT, IMG_WIDTH), is_train=True)
val_gen = TerraSegGenerator(val_ids, TRAIN_IMG_DIR, TRAIN_MASK_DIR, BATCH_SIZE, (IMG_HEIGHT, IMG_WIDTH), is_train=True)

model = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=1, activation='sigmoid')

callbacks = [
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_iou_score', mode='max', verbose=1),
    ReduceLROnPlateau(monitor='val_iou_score', factor=0.5, patience=3, min_lr=1e-6, verbose=1, mode='max'),
    EarlyStopping(monitor='val_iou_score', patience=8, restore_best_weights=True, verbose=1, mode='max')
]

model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score]
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks
)

print("\nGenerating Submission")

def rle_encode(img):
    pixels = img.flatten(order='F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

if os.path.exists('best_model.keras'):
    print("Loading Model")
    model.load_weights('best_model.keras')
else:
    print("Error")

test_ids = sorted(os.listdir(TEST_IMG_DIR))
submission_data = []

print("Processing")

for img_id in tqdm(test_ids):
    path = os.path.join(TEST_IMG_DIR, img_id)
    img = cv2.imread(path)
    if img is None: continue
        
    orig_h, orig_w = img.shape[:2]
    
=    img_resized = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    img_batch = preprocess_input(np.array([img_resized], dtype=np.float32))
    
    pred_batch = model.predict(img_batch, verbose=0)
    pred_mask = pred_batch[0]
    
    pred_binary = (pred_mask > 0.5).astype(np.uint8)
    final_mask = cv2.resize(pred_binary, (orig_w, orig_h), interpolation=cv2.INTER_NEAREST)
    
    rle = rle_encode(final_mask)
    clean_id = img_id.replace('.png', '') 
    
    submission_data.append([clean_id, rle])

df_sub = pd.DataFrame(submission_data, columns=['image_id', 'encoded_pixels'])
df_sub.to_csv('submission.csv', index=False)

print("\nsubmission.csv generated with")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.2 MB/s eta 0:00:00
Segmentation Models: using `tf.keras` framework.
✅ Configuration Set. Training on: /kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images
85521592/85521592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
🚀 Starting Training...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step - iou_score: 0.5701 - loss: 1.0085
Epoch 1: val_iou_score improved from -inf to 0.67832, saving model to best_model.keras
159/159 ━━━━━━━━━━━━━━━━━━━━ 143s 625ms/step - iou_score: 0.5706 - loss: 1.0070 - val_iou_score: 0.6783 - val_loss: 0.7228 - learning_rate: 1.0000e-04
Epoch 2/3
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 408ms/step - iou_score: 0.7972 - loss: 0.4405
Epoch 2: val_iou_score improved from 0.67832 to 0.82715, saving model to best_model.keras
159/159 ━━━━━━━━━━━━━━━━━━━━ 80s 504ms/step - iou_score: 0.7973 - loss: 0.4403 - val_iou_score: 0.8272 - val_loss: 0.3710 - learning_rate: 1.0000e-04
Epoch 3/3
159/159 ━━━━━━━━━━━━━━━━━━━━ 0s 365ms/step - iou_score: 0.8629 - loss: 0.3013
Epoch 3: val_iou_score improved from 0.82715 to 0.89126, saving model to best_model.keras
159/159 ━━━━━━━━━━━━━━━━━━━━ 74s 464ms/step - iou_score: 0.8630 - loss: 0.3011 - val_iou_score: 0.8913 - val_loss: 0.2411 - learning_rate: 1.0000e-04
Restoring model we

100%|██████████| 1002/1002 [01:35<00:00, 10.44it/s]



🎉 DONE! 'submission.csv' generated with 1002 rows.
Sample IDs: ['0001', '0002', '0003', '0004', '0005']
